# CHAPTER 9 - Quantum Support Vector Machines - Code

*Note*: You may skip the following four cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.

In [1]:
#!pip install scikit-learn
#!pip install pennylane==0.26
#!pip install qiskit==0.39.2
#!pip install qiskit_machine_learning==0.5.0

In [2]:
import numpy as np

print(np.__version__)
seed = 1234
np.random.seed(seed)

2.0.1


In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load diabetes data
df = pd.read_csv("diabetes.csv")

print(df.head())

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [4]:
x = df.drop("Outcome", axis=1)
y = df["Outcome"]

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(614, 8) (154, 8) (614,) (154,)


In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_tr = scaler.fit_transform(x_train)
x_te = scaler.transform(x_test)
print(x_tr.max(axis=0))
print(x_tr.min(axis=0))
print(x_te.max(axis=0))
print(x_te.min(axis=0))

[4.00455182 2.44134623 2.84279725 2.7624583  6.58309754 4.54036251
 5.79617534 4.18414491]
[-1.13052335 -3.77566322 -3.75268255 -1.32277365 -0.70120553 -4.13525578
 -1.16221007 -1.03594038]
[2.79629884 2.37886372 2.41030677 5.09687656 4.28415082 2.7431778
 2.7359119  2.96612501]
[-1.13052335 -1.8387055  -3.75268255 -1.32277365 -0.70120553 -4.13525578
 -1.09090296 -1.03594038]


In [10]:
import pennylane as qml
print(qml.__version__)

nqubits = 4
dev = qml.device("lightning.qubit", wires = nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AmplitudeEmbedding(
        a, wires=range(nqubits), pad_with=0, normalize=True)
    qml.adjoint(qml.AmplitudeEmbedding(
        b, wires=range(nqubits), pad_with=0, normalize=True))
    return qml.probs(wires = range(nqubits))

0.42.0


In [12]:
from sklearn.svm import SVC
def qkernel(A, B):
    return np.array([[kernel_circ(a, b)[0] for b in B] for a in A])

svm = SVC(kernel = qkernel).fit(x_tr, y_train)

In [14]:
from sklearn.metrics import accuracy_score

print(accuracy_score(svm.predict(x_te), y_test))

0.6623376623376623


In [15]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 8)

xs_tr = pca.fit_transform(x_tr)
xs_test = pca.transform(x_test)

c:\Users\leqian\anaconda3\envs\qiskit\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


In [21]:
nqubits = 8
dev = qml.device("lightning.qubit", wires=nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AngleEmbedding(a, wires=range(nqubits))
    qml.adjoint(qml.AngleEmbedding(b, wires=range(nqubits)))
    return qml.probs(wires = range(nqubits))

In [25]:
print(x_tr.shape, y_test.shape)

(614, 8) (154,)


In [27]:
svm = SVC(kernel = qkernel).fit(x_tr, y_train)

print(accuracy_score(svm.predict(x_te), y_test))

0.7077922077922078


In [28]:
from itertools import combinations

def ZZFeatureMap(nqubits, data):

    # Number of variables that we will load:
    # could be smaller than the number of qubits.
    nload = min(len(data), nqubits) 

    for i in range(nload):
        qml.Hadamard(i)
        qml.RZ(2.0 * data[i], wires = i)

    for pair in list(combinations(range(nload), 2)):
        q0 = pair[0]
        q1 = pair[1]

        qml.CZ(wires = [q0, q1])
        qml.RZ(2.0 * (np.pi - data[q0]) *
            (np.pi - data[q1]), wires = q1)
        qml.CZ(wires = [q0, q1])

In [31]:
nqubits = 4
dev = qml.device("lightning.qubit", wires = nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    ZZFeatureMap(nqubits, a)
    qml.adjoint(ZZFeatureMap)(nqubits, b)
    return qml.probs(wires = range(nqubits))

svm = SVC(kernel=qkernel).fit(x_tr, y_train)
print(accuracy_score(svm.predict(x_te), y_test))

0.6298701298701299


In [32]:
from sklearn.metrics import accuracy_score

In [33]:
from qiskit import *
import qiskit
print(qiskit.__version__)

1.2.4


In [34]:
from qiskit.circuit import Parameter
parameter = Parameter("x")
qc = QuantumCircuit(1)
qc.rx(parameter, 0)

In [35]:
from qiskit.circuit import ParameterVector
parameter = ParameterVector("x", length = 2)
qc = QuantumCircuit(2)
qc.rx(parameter[0], 0)
qc.rx(parameter[1], 1)

In [36]:
from qiskit.circuit.library import ZZFeatureMap #Update
#from qiskit.circuit.library import zz_feature_map #Update

zzfm = ZZFeatureMap(8)

In [37]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
import qiskit_machine_learning
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
print(qiskit_machine_learning.__version__)

0.8.3


In [38]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_aer import AerSimulator

sampler = Sampler()
fidelity_object = ComputeUncompute(sampler=sampler)

qkernel = FidelityQuantumKernel(feature_map = zzfm, fidelity = fidelity_object) #update

In [ ]:
from qiskit_machine_learning.algorithms import QSVC

qsvm = QSVC(quantum_kernel = qkernel, C=10)
qsvm.fit(x_tr, y_train)

In [ ]:
print(accuracy_score(qsvm.predict(x_te), y_test))

0.46153846153846156


*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [54]:
ibm_token = "1234"
IBMQ.save_account(ibm_token)

NameError: name 'IBMQ' is not defined

In [ ]:
provider = IBMQ.load_account()

In [ ]:
from qiskit.providers.ibmq import *

dev_list = provider.backends(
    filters = lambda x: x.configuration().n_qubits >= 7,
    simulator = False)
dev = least_busy(dev_list)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 7)

xss_tr = pca.fit_transform(x_tr)

xss_test = pca.transform(x_test)

In [ ]:
zzfm = ZZFeatureMap(7)
qkernel = QuantumKernel(feature_map = zzfm, quantum_instance = dev)
qsvm = QSVC(quantum_kernel = qkernel)
qsvm.fit(xss_tr, y_tr)